In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

with open('data/raw/crime.html') as f:
    soup = BeautifulSoup(f.read(), 'lxml')
    table = soup.find_all('table')[0]
    data_columns = table.find_all('th')
    column_data = table.find_all('tr')

    cols_to_use = [name.text.strip() for name in data_columns]

    cols_to_use.remove('Yearly Crime Rates per 100,000 people')
    cols_to_use.remove('Violent crime')
    cols_to_use.remove('Property crime')

    cols_to_use[3] = 'Total (Violent and Property)'
    cols_to_use[9] = "Total Violent"
    cols_to_use[13] = "Total Property"

    element = cols_to_use.pop(4)
    cols_to_use.insert(13, element)
    
    crime_data = pd.DataFrame(columns = cols_to_use)

    for row in column_data[3:]:
        row_data = row.find_all('td')
        individual_row_data = [data.text.strip() for data in row_data]
        crime_data.loc[len(crime_data)] = individual_row_data

    crime_data.at[0, 'City'] = 'Mobile'
    crime_data.at[26, 'City'] = 'Stockton'
    crime_data.at[73, 'City'] = 'Toledo'
    crime_data.at[81, 'City'] = 'Arlington'
    crime_data.at[47, 'City'] = 'Louisville Metro'
    
    new_columns = {'Murder andNonnegligentmanslaughter': 'Murder/Manslaughter', 
               'Rape1': 'Rape',
               'Aggravatedassault': 'Assault',
               'Larceny-theft': 'Larceny/Theft',
               'Motorvehicletheft': 'Vehicle Theft',
               'Arson2' : 'Arson'}
    crime_data.rename(columns = new_columns, inplace=True)
    
    crime_data['Population'] = pd.to_numeric(crime_data['Population'].replace(',', '', regex=True), errors='coerce')
    crime_data['Total Property'] = pd.to_numeric(crime_data['Total Property'].replace(',', '', regex=True), errors='coerce')
    crime_data['Larceny/Theft'] = pd.to_numeric(crime_data['Larceny/Theft'].replace(',', '', regex=True), errors='coerce')
    crime_data['Burglary'] = pd.to_numeric(crime_data['Burglary'].replace(',', '', regex=True), errors='coerce')
    crime_data['Vehicle Theft'] = pd.to_numeric(crime_data['Vehicle Theft'].replace(',', '', regex=True), errors='coerce')
    crime_data['Assault'] = pd.to_numeric(crime_data['Assault'].replace(',', '', regex=True), errors='coerce')
    crime_data['Rape'] = pd.to_numeric(crime_data['Rape'].replace(',', '', regex=True), errors='coerce')
    crime_data['Arson'] = pd.to_numeric(crime_data['Arson'].replace(',', '', regex=True), errors='coerce')
    crime_data['Total Violent'] = pd.to_numeric(crime_data['Total Violent'].replace(',', '', regex=True), errors='coerce')

    crime_data['Total (Violent and Property)'] = pd.to_numeric(crime_data['Total (Violent and Property)'], errors='coerce')
    crime_data['Murder/Manslaughter'] = pd.to_numeric(crime_data['Murder/Manslaughter'], errors='coerce')
    crime_data['Robbery'] = pd.to_numeric(crime_data['Robbery'], errors='coerce')
    
    crime_data.to_csv('crime_data.csv')